<a href="https://colab.research.google.com/github/CodeHunterOfficial/Practical-Time-Series-Analysis/blob/main/Chapter2_Finding_and_Wrangling_Time_Series_Data_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import io
import requests

In [4]:
YearJoined = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/year_joined.csv")

In [5]:
emails = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/emails.csv")

In [6]:
donations = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/donations.csv")

In [7]:
# 55 page
YearJoined.groupby('user').count().groupby('userStats').count()

,yearJoined
userStats,
1,1000


In [10]:
# 56 page
emails[emails.emailsOpened < 1]

,emailsOpened,user,week


In [11]:
emails[emails.user == 998]

,emailsOpened,user,week
25464,1.0,998.0,2017-12-04 00:00:00
25465,3.0,998.0,2017-12-11 00:00:00
25466,3.0,998.0,2017-12-18 00:00:00
25467,3.0,998.0,2018-01-01 00:00:00
25468,3.0,998.0,2018-01-08 00:00:00
25469,2.0,998.0,2018-01-15 00:00:00
25470,3.0,998.0,2018-01-22 00:00:00
25471,2.0,998.0,2018-01-29 00:00:00
25472,3.0,998.0,2018-02-05 00:00:00
25473,3.0,998.0,2018-02-12 00:00:00


In [12]:
#57 page
(pd.to_datetime(max(emails[emails.user== 998].week))-pd.to_datetime(min(emails[emails.user==998].week))).days/7

25.0

In [13]:
# 57 page
import datetime
date_time_str_max = max(emails[emails.user == 998].week)
date_time_str_min = min(emails[emails.user == 998].week)

date_time_obj_max = datetime.datetime.strptime(date_time_str_max, '%Y-%m-%d %H:%M:%S')
date_time_obj_min = datetime.datetime.strptime(date_time_str_min, '%Y-%m-%d %H:%M:%S')

(date_time_obj_max - date_time_obj_min).days/7

25.0

In [14]:
emails[emails.user == 998].shape

(24, 3)

In [15]:
 # 58
complete_idx = pd.MultiIndex.from_product((set(emails.week), set(emails.user)))

In [16]:
all_email = emails.set_index(['week', 'user']).reindex(complete_idx, fill_value=0).reset_index()
all_email.columns = ['week', 'user', 'emailsOpened']

In [17]:
all_email[all_email.user == 998].sort_values('week')

,week,user,emailsOpened
71147,2015-02-09 00:00:00,998.0,0.0
2155,2015-02-16 00:00:00,998.0,0.0
5389,2015-02-23 00:00:00,998.0,0.0
33956,2015-03-02 00:00:00,998.0,0.0
40424,2015-03-09 00:00:00,998.0,0.0
...,...,...,...
21559,2018-04-30 00:00:00,998.0,3.0
14552,2018-05-07 00:00:00,998.0,3.0
70069,2018-05-14 00:00:00,998.0,3.0
31800,2018-05-21 00:00:00,998.0,3.0


In [18]:
# 59 page
cutoff_dates = emails.groupby('user').week.agg(['min', 'max']).reset_index()
cutoff_dates = cutoff_dates.reset_index()

In [19]:
import warnings
warnings.filterwarnings('ignore')

for _, row in cutoff_dates.iterrows():
  user        = row['user']
  start_date  = row['min']
  end_date    = row['max']
  all_email.drop(all_email[all_email.user == user][all_email.week < start_date].index, inplace=True)
  all_email.drop(all_email[all_email.user == user][all_email.week > end_date].index, inplace=True)

In [20]:
# 61 page
donations.timestamp = pd.to_datetime(donations.timestamp)
donations.set_index('timestamp', inplace=True)
agg_don = donations.groupby('user').apply(lambda df: df.amount.resample("W-MON").sum().dropna())

In [21]:
agg_don

user   timestamp 
0.0    2015-03-30      25.0
       2015-04-06       0.0
       2015-04-13       0.0
       2015-04-20       0.0
       2015-04-27       0.0
                      ...  
995.0  2017-09-11       0.0
       2017-09-18       0.0
       2017-09-25       0.0
       2017-10-02    1000.0
998.0  2018-01-08      50.0
Name: amount, Length: 32352, dtype: float64

In [38]:
#61 page
merged_df = pd.DataFrame()

for user, user_email in all_email.groupby('user'):
  user_donations = agg_don[agg_don.index.get_level_values('user') == user]
  
  user_donations = user_donations.droplevel(0)
#  user_donations.set_index('timestamp', inplace = True)  
  
  user_email = all_email[all_email.user == user]
  user_email.sort_values('week', inplace=True)
  user_email.set_index('week', inplace=True)

  df = pd.merge(user_email, user_donations, how='left', left_index=True, right_index=True)
  df.fillna(0)

  merged_df = merged_df.append(df.reset_index()[['user', 'week', 'emailsOpened', 'amount']])



  ## Python
#for member, member_email in all_email.groupby ('user'):
 # member_donations = agg_don[agg_don.user==member]


In [27]:
merged_df.head()

,user,week,emailsOpened,amount
0,1.0,2015-06-29 00:00:00,3.0,NaN
1,1.0,2015-07-06 00:00:00,0.0,NaN
2,1.0,2015-07-13 00:00:00,2.0,NaN
3,1.0,2015-07-20 00:00:00,2.0,NaN
4,1.0,2015-07-27 00:00:00,3.0,NaN


In [33]:
#62 page
df = merged_df[merged_df.user == 998]
df['target'] = df.amount.shift(1)

df = df.fillna(0)
df.head(7)

#df = merged_df[merged_df.user == 998]
#df['target'] = df.amount.shift(1)
#df = df.fillna(0)
df

,user,week,emailsOpened,amount,target
0,998.0,2017-12-04 00:00:00,1.0,0.0,0.0
1,998.0,2017-12-11 00:00:00,3.0,0.0,0.0
2,998.0,2017-12-18 00:00:00,3.0,0.0,0.0
3,998.0,2017-12-25 00:00:00,0.0,0.0,0.0
4,998.0,2018-01-01 00:00:00,3.0,0.0,0.0
5,998.0,2018-01-08 00:00:00,3.0,0.0,0.0
6,998.0,2018-01-15 00:00:00,2.0,0.0,0.0
7,998.0,2018-01-22 00:00:00,3.0,0.0,0.0
8,998.0,2018-01-29 00:00:00,2.0,0.0,0.0
9,998.0,2018-02-05 00:00:00,3.0,0.0,0.0


In [ ]:
# page 89
air = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/AirPassengers.csv",
                  names=['Date','Passengers'])

In [ ]:
air

,Date,Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121
...,...,...
139,1960-08,606
140,1960-09,508
141,1960-10,461
142,1960-11,390


In [ ]:
#89 page
air['Smooth.5'] = air.ewm(alpha = .5).Passengers.mean()
air['Smooth.9'] = air.ewm(alpha = .9).Passengers.mean()

In [ ]:
air

,Date,Passengers,Smooth.5,Smooth.9
0,1949-01,112,112.000000,112.000000
1,1949-02,118,116.000000,117.454545
2,1949-03,132,125.142857,130.558559
3,1949-04,129,127.200000,129.155716
4,1949-05,121,124.000000,121.815498
...,...,...,...,...
139,1960-08,606,582.096411,606.665454
140,1960-09,508,545.048205,517.866545
141,1960-10,461,503.024103,466.686655
142,1960-11,390,446.512051,397.668665


In [ ]:
# 96 page
import datetime
datetime.datetime.utcnow()

datetime.datetime(2022, 8, 20, 13, 25, 2, 457496)

In [ ]:
datetime.datetime.now()

datetime.datetime(2022, 8, 20, 13, 26, 7, 169416)

In [ ]:
datetime.datetime.now(datetime.timezone.utc)

datetime.datetime(2022, 8, 20, 13, 26, 17, 119456, tzinfo=datetime.timezone.utc)

In [ ]:
#97 page
import pytz

western = pytz.timezone('US/Pacific')
western.zone

'US/Pacific'

In [ ]:
loc_dt = western.localize(datetime.datetime(2018, 5, 15, 12, 34, 0))
loc_dt

datetime.datetime(2018, 5, 15, 12, 34, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

In [ ]:
london_tz = pytz.timezone('Europe/London')
london_dt = loc_dt.astimezone(london_tz)
london_dt

datetime.datetime(2018, 5, 15, 20, 34, tzinfo=<DstTzInfo 'Europe/London' BST+1:00:00 DST>)

In [ ]:
f = '%Y-%m-%d %H:%M:%S %Z%z'
datetime.datetime(2018, 5, 12, 12, 15, 0, tzinfo = london_tz).strftime(f)

'2018-05-12 12:15:00 LMT-0001'

In [ ]:
event1 = datetime.datetime(2018, 5, 12, 12, 15, 0, tzinfo = london_tz)
event2 = datetime.datetime(2018, 5, 13, 9, 15, 0, tzinfo = western)
event2 - event1

datetime.timedelta(days=1, seconds=17520)

In [ ]:
#98 page
event1 = london_tz.localize(datetime.datetime(2018, 5, 12, 12, 15, 0))
event2 = western.localize(datetime.datetime(2018, 5, 13, 9, 15, 0))
event2 - event1

datetime.timedelta(days=1, seconds=18000)

In [ ]:
event1 = london_tz.localize((datetime.datetime(2018, 5, 12, 12, 15, 0))).astimezone(datetime.timezone.utc)
event2 = western.localize(datetime.datetime(2018, 5, 13, 9, 15, 0)).astimezone(datetime.timezone.utc)
event2 - event1

datetime.timedelta(days=1, seconds=18000)

In [ ]:
pytz.common_timezones

['Africa/Abidjan', 'Africa/Accra', 'Africa/Addis_Ababa', 'Africa/Algiers', 'Africa/Asmara', 'Africa/Bamako', 'Africa/Bangui', 'Africa/Banjul', 'Africa/Bissau', 'Africa/Blantyre', 'Africa/Brazzaville', 'Africa/Bujumbura', 'Africa/Cairo', 'Africa/Casablanca', 'Africa/Ceuta', 'Africa/Conakry', 'Africa/Dakar', 'Africa/Dar_es_Salaam', 'Africa/Djibouti', 'Africa/Douala', 'Africa/El_Aaiun', 'Africa/Freetown', 'Africa/Gaborone', 'Africa/Harare', 'Africa/Johannesburg', 'Africa/Juba', 'Africa/Kampala', 'Africa/Khartoum', 'Africa/Kigali', 'Africa/Kinshasa', 'Africa/Lagos', 'Africa/Libreville', 'Africa/Lome', 'Africa/Luanda', 'Africa/Lubumbashi', 'Africa/Lusaka', 'Africa/Malabo', 'Africa/Maputo', 'Africa/Maseru', 'Africa/Mbabane', 'Africa/Mogadishu', 'Africa/Monrovia', 'Africa/Nairobi', 'Africa/Ndjamena', 'Africa/Niamey', 'Africa/Nouakchott', 'Africa/Ouagadougou', 'Africa/Porto-Novo', 'Africa/Sao_Tome', 'Africa/Tripoli', 'Africa/Tunis', 'Africa/Windhoek', 'America/Adak', 'America/Anchorage', 'Amer

In [ ]:
pytz.country_timezones('RU')

['Europe/Kaliningrad',
 'Europe/Moscow',
 'Europe/Kirov',
 'Europe/Volgograd',
 'Europe/Astrakhan',
 'Europe/Saratov',
 'Europe/Ulyanovsk',
 'Europe/Samara',
 'Asia/Yekaterinburg',
 'Asia/Omsk',
 'Asia/Novosibirsk',
 'Asia/Barnaul',
 'Asia/Tomsk',
 'Asia/Novokuznetsk',
 'Asia/Krasnoyarsk',
 'Asia/Irkutsk',
 'Asia/Chita',
 'Asia/Yakutsk',
 'Asia/Khandyga',
 'Asia/Vladivostok',
 'Asia/Ust-Nera',
 'Asia/Magadan',
 'Asia/Sakhalin',
 'Asia/Srednekolymsk',
 'Asia/Kamchatka',
 'Asia/Anadyr']

In [ ]:
pytz.country_timezones('fr')

['Europe/Paris']

In [ ]:
ambig_time = western.localize(datetime.datetime(2002, 10, 27, 1, 30, 00)).astimezone(datetime.timezone.utc)
ambig_time_earlier = ambig_time - datetime.timedelta(hours=1)
ambig_time_later = ambig_time + datetime.timedelta(hours=1)

In [ ]:
#99 page
ambig_time_earlier.astimezone(western)

datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

In [ ]:
ambig_time.astimezone(western)

datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)

In [ ]:
ambig_time_later.astimezone(western)

datetime.datetime(2002, 10, 27, 2, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)

In [ ]:
ambig_time = western.localize(datetime.datetime(2002, 10, 27, 1, 30, 00), is_dst = True).astimezone(datetime.timezone.utc)
ambig_time_earlier = ambig_time - datetime.timedelta(hours=1)
ambig_time_later = ambig_time + datetime.timedelta(hours=1)

In [ ]:
ambig_time_earlier.astimezone(western)

datetime.datetime(2002, 10, 27, 0, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

In [ ]:
ambig_time.astimezone(western)

datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

In [ ]:
ambig_time_later.astimezone(western)

datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)